### 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import time
from glob import glob
from tqdm import tqdm
from scipy import interpolate

In [3]:
# conda install tensorflow

In [4]:
# conda install python=3.7

In [5]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

CNN으로 전처리해볼 것
1. 유량 3개, 수위 4개
2. 팔당댐 데이터

### 전체 수위 데이터셋 구축

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 데이터 불러오기 
전체df = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')
# for colab
# /content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv
# for jupyter
# /Volumes/GoogleDrive/.shortcut-targets-by-id/1rUx7_7LmBqAQTbWEWVTGvw0_d0v6N2ZT/신입기수 프로젝트/Dataset/rawdata.csv

In [48]:
행주대교_유량 = 전체df['행주대교_유량']

In [49]:


nanIndex = 행주대교_유량[행주대교_유량.isnull()].index

In [50]:
nanIndex

Int64Index([140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620,
            140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628,
            140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636,
            140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644,
            140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652,
            140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660,
            140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668,
            140669, 140670, 236080],
           dtype='int64')

In [51]:
consecutive = list(nanIndex)[:-1]
unconsecutive = list(nanIndex)[-1]
print(consecutive)
print(unconsecutive)

[140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620, 140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628, 140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636, 140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644, 140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652, 140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660, 140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668, 140669, 140670]
236080


In [12]:
# 58개 결측치 앞 인덱스까지 인풋 생성
x = 행주대교_유량.loc[:140612]

# 결측치로 아웃풋 생성
null = 행주대교_유량[140613:140671]

In [13]:
null

,행주대교_유량
140613,NaN
140614,NaN
140615,NaN
140616,NaN
140617,NaN
140618,NaN
140619,NaN
140620,NaN
140621,NaN
140622,NaN


In [14]:
x

,행주대교_유량
0,540.18
1,540.18
2,540.18
3,552.17
4,564.29
...,...
140608,2385.34
140609,2410.83
140610,2359.99
140611,2359.99


In [15]:
# conda install scikit-learn

In [16]:
# train, test data 생성할 때 필요
from sklearn.model_selection import train_test_split
# 정규화할 때 필요
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [17]:
# input 정규화
x_regular = x.copy()

for i in range(len(x.columns)):
    scaler = StandardScaler()                       #StandardScaler() , MinMaxScaler() , RobustScaler()
    a=np.array(x_regular.iloc[:,i])
    tmp_x = scaler.fit_transform(a.reshape(-1,1))
    x_regular.iloc[:,i] = tmp_x

In [18]:
x_regular

,행주대교_유량
0,-0.397022
1,-0.397022
2,-0.397022
3,-0.385557
4,-0.373968
...,...
140608,1.367353
140609,1.391727
140610,1.343113
140611,1.343113


### 행주대교 유량

In [19]:
start = time.time()

y_predict = []

xtrain = []
ytrain = []

w =  432 # window size
n = 58 # 예측할 데이터 개수

# xtrain 생성
# for i in range(len(x_regular)-w):
#     xtrain.append(np.array(x_regular.iloc[i:i + w, :]).astype(float))

for i in tqdm(range(len(x_regular)-w-n)):
    xtrain.append(np.array(x_regular.iloc[i:i + w, 0]).astype(float))

100%|██████████| 140123/140123 [00:15<00:00, 8922.79it/s]


In [20]:
# ytrain 생성
# for i in range(w, len(x_regular)):
#     ytrain.append(np.array(x_regular.iloc[i, :]).astype(float))
ytrain = []

for i in tqdm(range(w, len(x_regular)-n)):
  ytrain.append(np.array(x_regular.iloc[i:i+n, 0]).astype(float))

100%|██████████| 140123/140123 [00:15<00:00, 8781.46it/s]


In [21]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
print(xtrain.shape)
print(ytrain.shape)

(140123, 432)
(140123, 58)


In [22]:
df_xtrain = pd.DataFrame(xtrain, columns = [i for i in range(432)])

In [23]:
df_ytrain = pd.DataFrame(ytrain, columns = [f'R{i}' for i in range(58)])

In [24]:
df_series = pd.concat([df_xtrain, df_ytrain], axis = 1)

In [25]:
df_series.shape

(140123, 490)

In [26]:
xtrain.reshape(140123, 432, 1)
ytrain.reshape(140123, 58, 1)

array([[[-0.43063339],
        [-0.43063339],
        [-0.44158208],
        ...,
        [-0.25100782],
        [-0.26387852],
        [-0.28924698]],

       [[-0.43063339],
        [-0.44158208],
        [-0.45240647],
        ...,
        [-0.26387852],
        [-0.28924698],
        [-0.31409909]],

       [[-0.44158208],
        [-0.45240647],
        [-0.45240647],
        ...,
        [-0.28924698],
        [-0.31409909],
        [-0.35040669]],

       ...,

       [[-0.05718242],
        [-0.07186993],
        [-0.07186993],
        ...,
        [ 1.31899704],
        [ 1.36735299],
        [ 1.39172699]],

       [[-0.07186993],
        [-0.07186993],
        [-0.08642357],
        ...,
        [ 1.36735299],
        [ 1.39172699],
        [ 1.34311286]],

       [[-0.07186993],
        [-0.08642357],
        [-0.08642357],
        ...,
        [ 1.39172699],
        [ 1.34311286],
        [ 1.34311286]]])

In [27]:
# label
labels_col = [f'R{i}' for i in range(58)]
labels = df_series[labels_col]
series = df_series.drop(labels_col, axis=1)
labels.values.shape

(140123, 58)

In [28]:
# x_train, x_val, y_train, y_val 분류
x_train, x_val, y_train, y_val = train_test_split(series, labels.values, test_size = 0.4, random_state = 0)

In [29]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,422,423,424,425,426,427,428,429,430,431
133922,-0.385557,-0.397022,-0.408354,-0.408354,-0.408354,-0.397022,-0.385557,-0.385557,-0.385557,-0.385557,...,-0.198215,-0.263879,-0.314099,-0.350407,-0.314099,-0.263879,-0.198215,-0.157260,-0.129320,-0.129320
32870,-0.441582,-0.441582,-0.452406,-0.452406,-0.463097,-0.463097,-0.473663,-0.484096,-0.484096,-0.494413,...,-0.494413,-0.494413,-0.494413,-0.504587,-0.504587,-0.504587,-0.504587,-0.514647,-0.514647,-0.514647
53652,-0.350407,-0.350407,-0.350407,-0.350407,-0.362254,-0.362254,-0.385557,-0.408354,-0.430633,-0.452406,...,-0.616828,-0.616828,-0.616828,-0.616828,-0.616828,-0.616828,-0.616828,-0.625358,-0.625358,-0.616828
2311,-0.534364,-0.534364,-0.544041,-0.563003,-0.572288,-0.572288,-0.572288,-0.572288,-0.572288,-0.581448,...,-0.534364,-0.534364,-0.534364,-0.534364,-0.534364,-0.534364,-0.524572,-0.524572,-0.524572,-0.524572
132218,-0.484096,-0.484096,-0.494413,-0.494413,-0.504587,-0.504587,-0.504587,-0.514647,-0.514647,-0.514647,...,-0.373968,-0.373968,-0.385557,-0.397022,-0.408354,-0.430633,-0.463097,-0.484096,-0.504587,-0.524572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,-0.198215,-0.198215,-0.184694,-0.184694,-0.184694,-0.171039,-0.171039,-0.171039,-0.171039,-0.171039,...,-0.042371,-0.027425,-0.027425,-0.027425,-0.027425,-0.027425,-0.012355,-0.012355,-0.027425,-0.057182
97639,0.519474,0.577603,0.636908,0.656940,0.656940,0.656940,0.636908,0.597244,0.538723,0.481379,...,-0.633753,-0.633753,-0.633753,-0.633753,-0.642025,-0.642025,-0.642025,-0.642025,-0.642025,-0.642025
95939,-0.524572,-0.524572,-0.524572,-0.524572,-0.534364,-0.534364,-0.544041,-0.553584,-0.563003,-0.572288,...,-0.642025,-0.616828,-0.608174,-0.608174,-0.608174,-0.599396,-0.599396,-0.599396,-0.599396,-0.599396
117952,0.229339,0.212318,0.195422,0.178659,0.162031,0.129156,0.112910,0.096797,0.080819,0.064965,...,0.352165,0.316422,0.388425,0.462532,0.519474,0.577603,0.617009,0.617009,0.617009,0.597244


In [30]:
# xtest = 

In [31]:
# 모델링 및 모델 학습

if 'model' in locals(): 
    del model 
if 'history' in locals():
    del history


#He_normal
He_normal = keras.initializers.he_normal(seed=None)

model = Sequential()
model.add(Conv1D(filters = 8, input_shape = (w, 1), kernel_size = 2, padding='causal', dilation_rate=2, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))  #0.2~0.5  0.8까지

model.add(Conv1D(filters = 16, kernel_size = 2, padding='causal', dilation_rate=4, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters = 32, kernel_size = 1))

model.add(Flatten())   #model.add(GlobalAveragePooling1D(pool_size=2))

model.add(Dense(50, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(58))


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

opt = tf.keras.optimizers.Adam(lr=0.005) #기본값 0.001
model.compile(optimizer=opt, loss=root_mean_squared_error, metrics =["accuracy"])
model.summary()



callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5),
                keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                                monitor='val_loss',
                                                save_best_only=True)]



history = model.fit(xtrain, ytrain, epochs=300, validation_data = (x_val, y_val), verbose=1, callbacks=callbacks, batch_size = 3742) #batch size..?


keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

keras_model_best.evaluate(x_val,y_val)

# y_predict.append(keras_model_best.predict(xtest))

print("time :", time.time() - start, "초")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 432, 8)            24        
                                                                 
 batch_normalization (BatchN  (None, 432, 8)           32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 432, 8)            0         
                                                                 
 dropout (Dropout)           (None, 432, 8)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 432, 16)           272       
                                                                 
 batch_normalization_1 (Batc  (None, 432, 16)          64        
 hNormalization)                                        

In [32]:
x_test = x[-432:]

In [33]:
x_test = np.array(x_test).reshape((1, 432, 1))

In [34]:
x_test.shape

(1, 432, 1)

In [95]:
y_predict = keras_model_best.predict(x_test)

In [98]:
y_predict = np.array(y_predict).reshape(58,)
y_predict.shape

(58,)

In [99]:
행주대교_유량.iloc[consecutive] = y_predict

In [101]:
행주대교_유량.iloc[consecutive]

140613    1661.633911
140614    1751.476440
140615    1688.048828
140616    1622.373657
140617    1765.315186
140618    1788.879395
140619    1709.516968
140620    1612.337402
140621    1664.187866
140622    1736.327515
140623    1609.831909
140624    1658.627563
140625    1676.281128
140626    1688.603271
140627    1580.685059
140628    1496.293701
140629    1574.220337
140630    1584.558350
140631    1547.362061
140632    1550.074829
140633    1514.572388
140634    1531.987427
140635    1473.749634
140636    1521.786133
140637    1321.090088
140638    1511.630249
140639    1439.286377
140640    1321.038574
140641    1353.497192
140642    1371.565308
140643    1266.914795
140644    1261.488403
140645    1309.933350
140646    1267.629150
140647    1191.854858
140648    1274.977051
140649    1184.330322
140650    1253.122925
140651    1202.937012
140652    1188.223145
140653    1116.315186
140654    1211.260742
140655    1204.695923
140656    1177.091797
140657    1149.250732
140658    

In [102]:
# unconsecutive 부분 보간

unconsecutive2 = [unconsecutive-1, unconsecutive, unconsecutive+1]
행주대교_유량[unconsecutive2] = 행주대교_유량[unconsecutive2].interpolate()

In [103]:
행주대교_유량.isnull().sum()

0

In [104]:
행주대교_유량.to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/CNN_전처리/행주대교_유량.csv', index = False, encoding = 'cp949')